In [1]:
cd ..

/home/alex/tmp/skres


In [2]:
import os, sys, logging

import pandas as pd

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

/tmp/ipykernel_328960/3392261473.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
df = pd.read_csv(
    "https://gist.githubusercontent.com/AlexandreGazagnes/9018022652ba0933dd39c9df8a600292/raw/0845ef4c2df4806bb05c8c7423dc75d93e37400f/titanic_train_raw_csv"
)

In [4]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [5]:
y = df.Survived
X = df.iloc[:, 2:]

In [6]:
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [7]:
X

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
886,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [8]:
X = X.select_dtypes(exclude="object")
X

,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500
...,...,...,...,...,...
886,2,27.0,0,0,13.0000
887,1,19.0,0,0,30.0000
888,3,NaN,1,2,23.4500
889,1,26.0,0,0,30.0000


In [9]:
pipe = Pipeline(
    [
        ("imputer", SimpleImputer()),
        ("scaler", StandardScaler()),
        ("estimator", DummyClassifier()),
    ]
)

In [10]:
param_grid = {
    "imputer__strategy": ["mean", "median", "most_frequent"],
    "scaler__with_mean": [True, False],
    "estimator__strategy": ["most_frequent", "stratified", "uniform"],
}

In [11]:
param_grid = {
    "imputer__strategy": ["mean", "median", "most_frequent"],
    "scaler__with_mean": [True, False],
    "estimator__strategy": ["most_frequent", "stratified", "uniform"],
}

grid = GridSearchCV(
    pipe,
    param_grid=param_grid,
    cv=5,
    refit=True,
    return_train_score=True,
    n_jobs=-1,
    verbose=2,
)

In [12]:
grid.fit(X, y)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] END estimator__strategy=most_frequent, imputer__strategy=mean, scaler__with_mean=True; total time=   0.0s
[CV] END estimator__strategy=most_frequent, imputer__strategy=mean, scaler__with_mean=True; total time=   0.0s
[CV] END estimator__strategy=most_frequent, imputer__strategy=mean, scaler__with_mean=False; total time=   0.0s
[CV] END estimator__strategy=most_frequent, imputer__strategy=median, scaler__with_mean=False; total time=   0.0s
[CV] END estimator__strategy=most_frequent, imputer__strategy=mean, scaler__with_mean=True; total time=   0.0s
[CV] END estimator__strategy=most_frequent, imputer__strategy=median, scaler__with_mean=True; total time=   0.0s
[CV] END estimator__strategy=most_frequent, imputer__strategy=mean, scaler__with_mean=False; total time=   0.0s
[CV] END estimator__strategy=most_frequent, imputer__strategy=median, scaler__with_mean=False; total time=   0.0s
[CV] END estimator__strategy=most_frequen

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('imputer', SimpleImputer()),
                                       ('scaler', StandardScaler()),
                                       ('estimator', DummyClassifier())]),
             n_jobs=-1,
             param_grid={'estimator__strategy': ['most_frequent', 'stratified',
                                                 'uniform'],
                         'imputer__strategy': ['mean', 'median',
                                               'most_frequent'],
                         'scaler__with_mean': [True, False]},
             return_train_score=True, verbose=2)

In [13]:
from skres import SkRes

res = SkRes(grid)
res

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_estimator__strategy,param_imputer__strategy,param_scaler__with_mean,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
0,0.0132,0.0016,0.0053,0.0016,most_frequent,mean,True,"{'estimator__strategy': 'most_frequent', 'impu...",0.6162,0.0023,1,0.6162,0.0006
1,0.0116,0.0021,0.0045,0.0008,most_frequent,mean,False,"{'estimator__strategy': 'most_frequent', 'impu...",0.6162,0.0023,1,0.6162,0.0006
2,0.0131,0.0014,0.0047,0.0009,most_frequent,median,True,"{'estimator__strategy': 'most_frequent', 'impu...",0.6162,0.0023,1,0.6162,0.0006
3,0.0120,0.0023,0.0047,0.0013,most_frequent,median,False,"{'estimator__strategy': 'most_frequent', 'impu...",0.6162,0.0023,1,0.6162,0.0006
4,0.0158,0.0009,0.0048,0.0002,most_frequent,most_frequent,True,"{'estimator__strategy': 'most_frequent', 'impu...",0.6162,0.0023,1,0.6162,0.0006
5,0.0158,0.0031,0.0037,0.0002,most_frequent,most_frequent,False,"{'estimator__strategy': 'most_frequent', 'impu...",0.6162,0.0023,1,0.6162,0.0006
8,0.0112,0.0016,0.0044,0.0009,stratified,median,True,"{'estimator__strategy': 'stratified', 'imputer...",0.5500,0.0406,7,0.5429,0.0220
6,0.0101,0.0018,0.0051,0.0026,stratified,mean,True,"{'estimator__strategy': 'stratified', 'imputer...",0.5320,0.0232,8,0.5062,0.0113
7,0.0102,0.0011,0.0053,0.0015,stratified,mean,False,"{'estimator__strategy': 'stratified', 'imputer...",0.5241,0.0405,9,0.5202,0.0096
9,0.0133,0.0038,0.0064,0.0038,stratified,median,False,"{'estimator__strategy': 'stratified', 'imputer...",0.5219,0.0289,10,0.5440,0.0201
